In [ ]:
pip install pandas
pip install numpy
pip install seaborn
pip install bertopic

In [1]:
import re
import pandas as pd
import json
import numpy as np
import seaborn as sns
from bertopic import BERTopic

/home/rodrigo98rm/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-08-23 21:09:03.501598: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-23 21:09:03.501614: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def df_tweets_candidatos(json_filename):
    df = pd.read_json(json_filename).drop(columns = ['replies'])
    return df

tweets = df_tweets_candidatos('./datasets/dataset.json')


# Adicionar nomes dos candidatos no dataframe
for index, row in tweets.iterrows():
    candidato = ''
    match row.author_id:
        case 2670726740:
            candidato = 'lula'
        case 128372940:
            candidato = 'bolsonaro'
        case 33374761:
            candidato = 'ciro'
        case _:
            candidato: 'n/d'
    tweets.at[index, 'candidato'] = candidato

# Embaralhar tweets
tweets = tweets.sample(frac=1)
        

print(f'Número de tweets: {len(tweets)}\n')
print(tweets.head())

Número de tweets: 300

      author_id      conversation_id                created_at  \
279   128372940  1556574838508208128 2022-08-08 09:35:36+00:00   
27   2670726740  1559863217819713536 2022-08-17 11:22:26+00:00   
253   128372940  1557892369990795264 2022-08-12 00:51:00+00:00   
144    33374761  1559701528994258944 2022-08-17 00:39:57+00:00   
75   2670726740  1558795209391095808 2022-08-14 12:38:33+00:00   

                                                  text                   id  \
279  - Também, a ampliação do Teto do financiamento...  1556574840429187072   
27   Uma das primeiras coisas que quero fazer é rec...  1559863217819713536   
253  - Do contrário, assinar uma carta pela democra...  1557892371613900800   
144  RT @ElvisGov12: É com muito orgulho que inicio...  1559701528994258944   
75   Um feliz dia dos pais. Com muito carinho, frat...  1558795209391095808   

     candidato  
279  bolsonaro  
27        lula  
253  bolsonaro  
144       ciro  
75        lula  


In [3]:
# create model 
model = BERTopic(language="multilingual", verbose=True, min_topic_size=3, top_n_words=10, calculate_probabilities=True)

# convert to list 
docs = tweets.text.to_list()
 
tweets_topic, probabilities = model.fit_transform(docs)

Batches: 100%|██████████| 10/10 [00:05<00:00,  1.82it/s]
2022-08-23 21:09:25,308 - BERTopic - Transformed documents to Embeddings
2022-08-23 21:09:29,661 - BERTopic - Reduced dimensionality
2022-08-23 21:09:29,688 - BERTopic - Clustered reduced embeddings


Adicionar no dataframe o tópico identificado para cada tweet

In [4]:

topic_names = model.generate_topic_labels(nr_words=4)

# Array indicando o tópico de cada tweet (numérico -> índice do array "topic_names" + 1)
# print(tweets_topic)
# Array com todos os tópicos nomeados
# print(topic_names)

for index, tweet_topic in enumerate(tweets_topic):
    tweets.at[index + 1, 'topico'] = topic_names[tweet_topic + 1]

tweets.head()


author_id  conversation_id                created_at  \
279  1.283729e+08     1.556575e+18 2022-08-08 09:35:36+00:00   
27   2.670727e+09     1.559863e+18 2022-08-17 11:22:26+00:00   
253  1.283729e+08     1.557892e+18 2022-08-12 00:51:00+00:00   
144  3.337476e+07     1.559702e+18 2022-08-17 00:39:57+00:00   
75   2.670727e+09     1.558795e+18 2022-08-14 12:38:33+00:00   

                                                  text            id  \
279  - Também, a ampliação do Teto do financiamento...  1.556575e+18   
27   Uma das primeiras coisas que quero fazer é rec...  1.559863e+18   
253  - Do contrário, assinar uma carta pela democra...  1.557892e+18   
144  RT @ElvisGov12: É com muito orgulho que inicio...  1.559702e+18   
75   Um feliz dia dos pais. Com muito carinho, frat...  1.558795e+18   

     candidato                           topico  
279  bolsonaro               0_https_co_ciro_rt  
27        lula               0_https_co_ciro_rt  
253  bolsonaro  8_ele_bolsonaro_porque_mentiras  
144       ciro                12_não_sou_que_eu  
75        lula         14_bolsonaro_jair_pai_pr

Abaixo, os tópicos identificados entre todos os tweets

In [5]:
freq = model.get_topic_info()
print(freq)

    Topic  Count                                     Name
0      -1     45                         -1_de_que_em_não
1       0     55                       0_https_co_ciro_rt
2       1     51                1_brasil_que_esperança_de
3       2     16               2_programa_de_renda_mínima
4       3     15                     3_08_2022_giro_https
5       4     15                    4_de_em_redução_menor
6       5     10               5_lula_https_co_equipelula
7       6     10                       6_ano_ton_safra_de
8       7      9                    7_eu_voto_eleição_não
9       8      8          8_ele_bolsonaro_porque_mentiras
10      9      7     9_sobre_ciro_cironorodaviva_responde
11     10      7                       10_deus_se_você_os
12     11      6        11_saudades_carta_democracia_cuba
13     12      6                        12_não_sou_que_eu
14     13      6             13_reeleição_política_um_que
15     14      6                 14_bolsonaro_jair_pai_pr
16     15     

Visualização de tweets com tópicos semelhantes:

In [6]:
model.visualize_documents(docs)

In [8]:
model.visualize_topics()

In [7]:
model.visualize_barchart(top_n_topics=5)

In [9]:
model.visualize_heatmap(width=750, height=750)

In [10]:
model.visualize_term_rank()

TODO:
- Normalize, remove stop words, lower case, etc